In [9]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
#os.environ['CUDA_LAUNCH_BLOCKING'] = str(1)
#os.environ["TORCH_USE_CUDA_DSA"]= str(0)
device = "cuda"

In [10]:
from __future__ import print_function

import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from torch.autograd import Variable
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score
from torchsummary import summary
import time

In [11]:
import import_ipynb
from Datagen import Data
from lstm1 import LSTM

In [12]:
def load_data():
    seglength = 300
    train_ratio = 0.8
    val_ratio = 0.1

    focal_train, focal_test, focal_train_labels, focal_test_labels = Data('D:\EEG\FOCAL', train_ratio, seglength, class_num = 1)
    ige_train, ige_test, ige_train_labels, ige_test_labels = Data('D:\EEG\IGE', train_ratio, seglength, class_num = 2)
    
    X_train = torch.cat((focal_train, ige_train))
    Y_train = torch.cat((focal_train_labels, ige_train_labels)).view(-1, 1)
    X_test = torch.cat((focal_train, ige_train))
    Y_test = torch.cat((focal_train_labels, ige_train_labels)).view(-1, 1)

    del focal_train, focal_test, focal_train_labels, focal_test_labels, ige_train, ige_test, ige_train_labels, ige_test_labels

    pnes_train, pnes_test, pnes_train_labels, pnes_test_labels = Data('D:\EEG\PNES', train_ratio, seglength, class_num = 3)

    X_train = torch.cat((X_train, pnes_train))
    Y_train = torch.cat((Y_train, pnes_train_labels.view(-1, 1)))
    X_test = torch.cat((X_test, pnes_test))
    Y_test = torch.cat((Y_test, pnes_test_labels.view(-1, 1)))

    del pnes_train, pnes_test, pnes_train_labels, pnes_test_labels

    temporal_train, temporal_test, temporal_train_labels, temporal_test_labels = Data('D:\EEG\TLE', train_ratio, seglength, class_num = 4)

    X_train = torch.cat((X_train, temporal_train))
    Y_train = torch.cat((Y_train, temporal_train_labels.view(-1, 1)))
    X_test = torch.cat((X_test, temporal_test))
    Y_test = torch.cat((Y_test, temporal_test_labels.view(-1, 1)))

    del temporal_train, temporal_test, temporal_train_labels, temporal_test_labels

    X_val = X_train[:int(val_ratio * len(Y_train))]
    Y_val = Y_train[:int(val_ratio * len(Y_train))]
    X_train = X_train[int(val_ratio * len(Y_train)):]
    Y_train = Y_train[int(val_ratio * len(Y_train)):].view(-1, 1)

    return X_train, Y_train, X_val, Y_val, X_test, Y_test

In [13]:
#X_train, Y_train, X_val, Y_val, X_test, Y_test = load_data()

In [14]:
#np.shape(X_train)

In [20]:

X_train = torch.randn([14231, 300, 18])
Y_train = torch.ones([14231, 1]) * 0.5
X_val = torch.randn([1581, 300, 18])
Y_val = torch.ones([1581, 1]) * 0.5
X_test = torch.randn([10249, 300, 18])
Y_test = torch.ones([10249, 1]) * 0.5


In [16]:
def evaluate(model, X, Y, params = ["fmeasure"]):
    torch.cuda.empty_cache()
    results = [] 
    predicted = []
    batch_size=1

    
    for i in range(int((len(X)/batch_size))):
        s = i*batch_size
        e = i*batch_size+batch_size
            
        inputs = Variable(X[s:e])
        pred = model(inputs)
            
        predicted.append(pred.data.cpu().numpy())
            
    predicted = torch.Tensor(predicted).view(-1, 1)

    for param in params:
        if param == 'acc':
            results.append(accuracy_score(Y, np.round(predicted)))
        if param == "auc":
            results.append(roc_auc_score(Y, predicted))
        if param == "recall":
            results.append(recall_score(Y, np.round(predicted)))
        if param == "precision":
            results.append(precision_score(Y, np.round(predicted)))
        if param == "fmeasure":
            precision = precision_score(Y, np.round(predicted))
            recall = recall_score(Y, np.round(predicted))
            results.append(2*precision*recall/ (precision+recall))
    return results



In [21]:

model = LSTM(1, 18, 20, 20).cuda()

model.forward(X_train[0:20].cuda())
#summary(model.cuda(), (1, 300, 18))
#model.cpu().forward(X_train[0:2])

tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

In [71]:
batch_size = 1024
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-2)
torch.cuda.empty_cache()

model.train()

for epoch in range(3):  # loop over the dataset multiple times
    print("\nEpoch ", epoch)
    running_loss = 0.0
    for i in range(int(len(X_train)/batch_size-1)):
        s = i*batch_size
        e = i*batch_size+batch_size
        
        inputs = Variable(X_train[s:e].cuda())
        labels = Variable(Y_train[s:e].cuda())



        optimizer.zero_grad()
        outputs = model.forward(inputs)

        loss = criterion(outputs, labels)
        loss.backward()

        optimizer.step()

        #print(loss.data)
    
    


Epoch  0

Epoch  1

Epoch  2


In [78]:
torch.cuda.empty_cache()
criterion(outputs, labels)
#evaluate(model, X_val.cuda(), Y_val.cpu(), params = ["fmeasure"])

tensor(50., device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)

In [72]:
model.forward(X_val.cuda()[2:20])

tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

In [83]:
x = 0
for i in model.parameters():
    x = x + 1
x

82